Group 125 - Christian Algaranaz, Krish Arora, Mike Min, Musa Sayeed

# How to Pick A Top-Performing Investment Portfolio Based On Different Economic Assumptions About the American Economy

## Introduction
___

The investment world is a vast place with countless financial products to choose from! Our project specifically focuses on investment portfolios/bundles composed of different equity and bond mutual funds;  a mutual fund is a type of investment vehicle that pools different assets, such as stocks or bonds. More specifically, we are interested in exploring the annual performance of 5 different portfolios, with each having a unique mutual funds or exchange traded fund composition (see diagram 1 below as reference), from 1987-2021. Based on these returns, we then pay attention to annual U.S. inflation, unemployment, and GDP growth rates data for the same time span. These explanatory variables will then be used to help predict and answer our project question: **Which investment portfolio is expected to earn the highest return under different US inflation, unemployment, and real GDP growth expectations?** A focus is given to U.S data since the American stock market accounts for ~60% of the world's total market capitalization.

Ther rationale behind choosing annual inflation, GDP growth, and unemployment data as predictors is because these indicators are a key economical factors that influence any economy. This changes in the economy gets reflected into financial markets, which ultimately impacts investment portfolios of every type. 

#*****A NEW DIAGRAM IN NEEDED IF WE DECIDE TO GO WITH THE NEW PORTFOLIOS (CHANGING THIS TEXT WOULD ALSO BE REQUIERED)*****

<img src="https://i.imgur.com/oOdGM3V.png"/>


### Loading and Merging the Data
___

In [1]:
library(tidyverse) #function that allows us to import our excel files in csv format

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [2]:
#importing inflation rate data
inflation_url <- "https://raw.githubusercontent.com/Arioniums/DSCI_100_125/main/inflation_data.csv"
inflation_path <- "inflation_data.csv"
download.file(inflation_url, destfile = inflation_path)
inflation_data <- read_csv(inflation_path, skip=11, col_names = c("date","annual_inflation"))

#reformatting the date values and filtering years to only 1997-2021
inflation_data$date <- format(as.Date(inflation_data$date,'%y-%m-%d'),'%Y')
inflation_data_c <- filter(inflation_data, between(date, 1997, 2021))

#renaming date column into year
names(inflation_data_c)[names(inflation_data_c) == 'date'] <- 'year'

#c stands for cleaned
inflation_data_c

Rows: 33 Columns: 2
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl  (1): annual_inflation
date (1): date

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


year,annual_inflation
<chr>,<dbl>
1997,2.3376899
1998,1.5522791
1999,2.1880272
2000,3.3768573
2001,2.8261711
2002,1.5860316
2003,2.2700950
2004,2.6772367
2005,3.3927468


In [3]:
#importing gdp growth rate data
real_gdp_url <- "https://raw.githubusercontent.com/Arioniums/DSCI_100_125/main/real_gdp_data.csv"
real_gdp_path <- "real_gdp_data.csv"
download.file(real_gdp_url, destfile = real_gdp_path)
real_gdp_data <- read_csv(real_gdp_path, skip=3, col_names = c("year","real_gdp_growth_rate","type"))

#reformatting the year values and filtering years to only 1997-2021
real_gdp_data$year <- format(as.Date(real_gdp_data$year, "'%y"),'%Y')
gdp_data_c <- select(real_gdp_data, year, real_gdp_growth_rate) |>
                filter(between(year, 1997, 2021))

#clarification of gdp growth rate unit (percentage)
names(gdp_data_c)[names(gdp_data_c) == 'real_gdp_growth_rate'] <- 'real_gdp_growth_rate_percentage'

#c stands for cleaned
gdp_data_c

Rows: 32 Columns: 3
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (2): year, type
dbl (1): real_gdp_growth_rate

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


year,real_gdp_growth_rate_percentage
<chr>,<dbl>
1997,4.4
1998,4.5
1999,4.8
2000,4.1
2001,1.0
2002,1.7
2003,2.8
2004,3.9
2005,3.5


In [4]:
#importing annual unemployment rate data
unemployment_url <- "https://raw.githubusercontent.com/Arioniums/DSCI_100_125/main/unemployment_rates_data.csv"
unemployment_path <- "unemployment_rates_data.csv"
download.file(unemployment_url, destfile = unemployment_path)
unemployment_data <- read_csv(unemployment_path, skip=1)

#filtering for USA unemployment data, selecting for TIME and Value columns, and then filtering years to only 1997-2021
unemployment_data_c <- filter(unemployment_data, LOCATION == "USA") |>
                    select(TIME, Value) |>
                    filter(between(TIME, 1997, 2021))

##renaming TIME column into year and Value column into annual_unemployment rate
names(unemployment_data_c)[names(unemployment_data_c) == 'TIME'] <- 'year'
names(unemployment_data_c)[names(unemployment_data_c) == 'Value'] <- 'annual_unemployment_rate'

#c stands for cleaned
unemployment_data_c

New names:
• `` -> `...9`
• `` -> `...10`
• `` -> `...11`
• `` -> `...12`
• `` -> `...13`
• `` -> `...14`
• `` -> `...15`
• `` -> `...16`
• `` -> `...17`
• `` -> `...18`
• `` -> `...19`
• `` -> `...20`
• `` -> `...21`
• `` -> `...22`
• `` -> `...23`
• `` -> `...24`
• `` -> `...25`
• `` -> `...26`
• `` -> `...27`
• `` -> `...28`
• `` -> `...29`
• `` -> `...30`
• `` -> `...31`
• `` -> `...32`
• `` -> `...33`
• `` -> `...34`
• `` -> `...35`
• `` -> `...36`
• `` -> `...37`
• `` -> `...38`
• `` -> `...39`
• `` -> `...40`
• `` -> `...41`
• `` -> `...42`
• `` -> `...43`
• `` -> `...44`
• `` -> `...45`
• `` -> `...46`
• `` -> `...47`
• `` -> `...48`
• `` -> `...49`
• `` -> `...50`
• `` -> `...51`
• `` -> `...52`
• `` -> `...53`
• `` -> `...54`
• `` -> `...55`
• `` -> `...56`
• `` -> `...57`
• `` -> `...58`
• `` -> `...59`
• `` -> `...60`
• `` -> `...61`
• `` -> `...62`
• `` -> `...63`
• `` -> `...64`
• `` -> `...65`
• `` -> `...66`
• `` -> `...67`
• `` -> `...68`
• `` -> `...69`
• `` -> `...70

year,annual_unemployment_rate
<dbl>,<dbl>
1997,4.950000
1998,4.508333
1999,4.216667
2000,3.991667
2001,4.733333
2002,5.775000
2003,5.991667
2004,5.533333
2005,5.066667


In [5]:
#importing income portfolio return data, selecting for year, income_portfolio_path, and filtering years to only 1997-2021
income_portfolio_url <- "https://raw.githubusercontent.com/Arioniums/DSCI_100_125/main/income_portfolio_data.csv"
income_portfolio_path <- "income_portfolio_data.csv"
download.file(income_portfolio_url, destfile = income_portfolio_path)
income_portfolio_data_c <- read_csv(income_portfolio_path, skip = 1, col_names= c("year","X2","income_portfolio_return")) |>
                            select(year, income_portfolio_return) |>
                            filter(between(year, 1997, 2021))

#deleting the percentage unit in the column income_portfolio_return and renaming the income_portfolio_return into income_portfolio_return_percentage
income_portfolio_data_c$income_portfolio_return = as.numeric(gsub("[\\%,]", "", income_portfolio_data_c$income_portfolio_return))
names(income_portfolio_data_c)[names(income_portfolio_data_c) == 'income_portfolio_return'] <- 'income_portfolio_return_percentage'

#c stands for cleaned
income_portfolio_data_c

Rows: 598 Columns: 11
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (8): year, X2, income_portfolio_return, X4, X5, X6, X7, X8
lgl (3): X9, X10, X11

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Warning message in between(year, 1997, 2021):
“NAs introduced by coercion”


year,income_portfolio_return_percentage
<chr>,<dbl>
1997,10.39
1998,11.30
1999,5.32
2000,4.77
2001,3.38
2002,4.77
2003,12.84
2004,7.36
2005,2.71


In [6]:
#importing sixty/forty and forty/sixty portfolio data
sixty_forty_and_forty_sixty_portfolio_url <- "https://raw.githubusercontent.com/Arioniums/DSCI_100_125/main/sixty_forty_and_forty_sixty_portfolio_data.csv"
                                              
sf_and_fs_path <- "sixty_forty_and_forty_sixty_portfolio_data.csv"
download.file(sixty_forty_and_forty_sixty_portfolio_url, destfile = sf_and_fs_path)

#selecting for year, 60/40_portfolio_return, 40/60_portfolio_return, and filtering years to only 1997-2021
sf_fs_portfolio_data_c <- read_csv(sf_and_fs_path, skip = 3, col_names= c("year","X2","60/40_portfolio_return","X3","40/60_portfolio_return")) |>
                                                select(year, "60/40_portfolio_return", "40/60_portfolio_return") |>
                                                filter(between(year, 1997, 2021))

#deleting the percentage unit in the column 60/40_portfolio_return and renaming the 60/40_portfolio_return into 60/40_portfolio_return_percentage
sf_fs_portfolio_data_c$'60/40_portfolio_return' = as.numeric(gsub("[\\%,]", "", sf_fs_portfolio_data_c$'60/40_portfolio_return'))
names(sf_fs_portfolio_data_c)[names(sf_fs_portfolio_data_c) == '60/40_portfolio_return'] <- '60/40_portfolio_return_percentage'

#deleting the percentage unit in the column 40/60_portfolio_return and renaming the 40/60_portfolio_return into 40/60_portfolio_return_percentage
sf_fs_portfolio_data_c$'40/60_portfolio_return' = as.numeric(gsub("[\\%,]", "", sf_fs_portfolio_data_c$'40/60_portfolio_return'))
names(sf_fs_portfolio_data_c)[names(sf_fs_portfolio_data_c) == '40/60_portfolio_return'] <- '40/60_portfolio_return_percentage'

#c stands for cleaned
sf_fs_portfolio_data_c

Rows: 769 Columns: 11
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (8): year, X2, 60/40_portfolio_return, X3, 40/60_portfolio_return, X6, X...
lgl (3): X9, X10, X11

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Warning message in between(year, 1997, 2021):
“NAs introduced by coercion”


year,60/40_portfolio_return_percentage,40/60_portfolio_return_percentage
<chr>,<dbl>,<dbl>
1997,22.37,18.06
1998,17.39,14.45
1999,13.98,9.07
2000,-1.79,2.61
2001,-3.21,0.67
2002,-9.27,-3.43
2003,20.40,14.93
2004,9.20,7.55
2005,4.55,3.83


In [7]:
#downloading growth, moderate, conservative portfolio data
gmc_url <- "https://raw.githubusercontent.com/Arioniums/DSCI_100_125/main/growth_moderate_conservative_portfolios.csv"
gmc_path <- "growth_moderate_conservative_portfolios.csv"
download.file(gmc_url, destfile = gmc_path)

#importing growth, moderate, conservative portfolio data, selecting for year, growth_portfolio_return, moderate_portfolio_return, conservative_portfolio_return, and filter to only 1997-2021
gmc_portfolios_data_c <- read_csv(gmc_path, skip = 3, col_names= c("year","X2","growth_portfolio_return","X3","moderate_portfolio_return","X4","conservative_portfolio_return")) |>
                                                select(year, growth_portfolio_return, moderate_portfolio_return, conservative_portfolio_return) |>
                                                filter(between(year, 1997, 2021))

#deleting the percentage unit in the column growth_portfolio_return and renaming the growth_portfolio_return into growth_portfolio_return_percentage
gmc_portfolios_data_c$'growth_portfolio_return' = as.numeric(gsub("[\\%,]", "", gmc_portfolios_data_c$'growth_portfolio_return'))
names(gmc_portfolios_data_c)[names(gmc_portfolios_data_c) == 'growth_portfolio_return'] <- 'growth_portfolio_return_percentage'

#deleting the percentage unit in the column moderate_portfolio_return and renaming the moderate_portfolio_return into moderate_portfolio_return_percentage
gmc_portfolios_data_c$'moderate_portfolio_return' = as.numeric(gsub("[\\%,]", "", gmc_portfolios_data_c$'moderate_portfolio_return'))
names(gmc_portfolios_data_c)[names(gmc_portfolios_data_c) == 'moderate_portfolio_return'] <- 'moderate_portfolio_return_percentage'

#deleting the percentage unit in the column conservative_portfolio_return and renaming the conservative_portfolio_return into conservative_portfolio_return_percentage
gmc_portfolios_data_c$'conservative_portfolio_return' = as.numeric(gsub("[\\%,]", "", gmc_portfolios_data_c$'conservative_portfolio_return'))
names(gmc_portfolios_data_c)[names(gmc_portfolios_data_c) == 'conservative_portfolio_return'] <- 'conservative_portfolio_return_percentage'

#c stands for cleaned
gmc_portfolios_data_c

Rows: 782 Columns: 16
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (12): year, X2, growth_portfolio_return, X3, moderate_portfolio_return, ...
lgl  (4): X13, X14, X15, X16

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Warning message in between(year, 1997, 2021):
“NAs introduced by coercion”


year,growth_portfolio_return_percentage,moderate_portfolio_return_percentage,conservative_portfolio_return_percentage
<chr>,<dbl>,<dbl>,<dbl>
1997,17.50,15.13,12.76
1998,18.26,15.94,13.62
1999,20.79,15.63,10.47
2000,-8.06,-3.78,0.49
2001,-9.79,-5.40,-1.01
2002,-12.99,-7.07,-1.15
2003,29.08,23.67,18.26
2004,13.41,11.40,9.38
2005,7.68,6.02,4.36


In [8]:
#merging all cleaned datasets
project_ds1 <- merge(
                    x = inflation_data_c, 
                    y = gdp_data_c,
                    by = "year")
project_ds2 <- merge(
                    x = project_ds1, 
                    y = unemployment_data_c,
                    by = "year")
project_ds3 <- merge(
                    x = project_ds2, 
                    y = income_portfolio_data_c,
                    by = "year")
project_ds4 <- merge(
                    x = project_ds3, 
                    y = sf_fs_portfolio_data_c,
                    by = "year")
project_ds_c <- merge(
                    x = project_ds4, 
                    y = gmc_portfolios_data_c,
                    by = "year")

#finding max portfolio return value and name
project_ds_c$max_portfolio_return_value <- do.call(pmax, project_ds_c[5:10])
project_ds_c$max_portfolio_return_name <- colnames(project_ds_c[5:10])[max.col(project_ds_c[5:10])]

#the dataset before selecting process above
project_ds_c

year,annual_inflation,real_gdp_growth_rate_percentage,annual_unemployment_rate,income_portfolio_return_percentage,60/40_portfolio_return_percentage,40/60_portfolio_return_percentage,growth_portfolio_return_percentage,moderate_portfolio_return_percentage,conservative_portfolio_return_percentage,max_portfolio_return_value,max_portfolio_return_name
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1997,2.3376899,4.4,4.950000,10.39,22.37,18.06,17.50,15.13,12.76,22.37,60/40_portfolio_return_percentage
1998,1.5522791,4.5,4.508333,11.30,17.39,14.45,18.26,15.94,13.62,18.26,growth_portfolio_return_percentage
1999,2.1880272,4.8,4.216667,5.32,13.98,9.07,20.79,15.63,10.47,20.79,growth_portfolio_return_percentage
2000,3.3768573,4.1,3.991667,4.77,-1.79,2.61,-8.06,-3.78,0.49,4.77,income_portfolio_return_percentage
2001,2.8261711,1.0,4.733333,3.38,-3.21,0.67,-9.79,-5.40,-1.01,3.38,income_portfolio_return_percentage
2002,1.5860316,1.7,5.775000,4.77,-9.27,-3.43,-12.99,-7.07,-1.15,4.77,income_portfolio_return_percentage
2003,2.2700950,2.8,5.991667,12.84,20.40,14.93,29.08,23.67,18.26,29.08,growth_portfolio_return_percentage
2004,2.6772367,3.9,5.533333,7.36,9.20,7.55,13.41,11.40,9.38,13.41,growth_portfolio_return_percentage
2005,3.3927468,3.5,5.066667,2.71,4.55,3.83,7.68,6.02,4.36,7.68,growth_portfolio_return_percentage


In [9]:
#select for year, annual_inflation, real_gdp_growth_rate, annual_unemployment_rate, max_portfolio_return_value, max_portfolio_return_name
project_ds <- select(project_ds_c, year, annual_inflation, real_gdp_growth_rate_percentage, annual_unemployment_rate, max_portfolio_return_value, max_portfolio_return_name)

#Final data set, years will be deleted for training and testing
project_ds

#Our table provides information about the top performing investment portfolios for given year from 1997 to 2021. The names of each portfolio was selected based on comparison analysis of 6 different portfolios.
#The very last column (max_portfolio_return_name) shows the name of protfolio with maximum return rate. 

#all of the variables are in yearly figures. It's measurements are in percentages.

#With the classification algorithm, we will be spliting using initial_split to split the data to be used for analysis and then to be used to for checking the code.

year,annual_inflation,real_gdp_growth_rate_percentage,annual_unemployment_rate,max_portfolio_return_value,max_portfolio_return_name
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1997,2.3376899,4.4,4.950000,22.37,60/40_portfolio_return_percentage
1998,1.5522791,4.5,4.508333,18.26,growth_portfolio_return_percentage
1999,2.1880272,4.8,4.216667,20.79,growth_portfolio_return_percentage
2000,3.3768573,4.1,3.991667,4.77,income_portfolio_return_percentage
2001,2.8261711,1.0,4.733333,3.38,income_portfolio_return_percentage
2002,1.5860316,1.7,5.775000,4.77,income_portfolio_return_percentage
2003,2.2700950,2.8,5.991667,29.08,growth_portfolio_return_percentage
2004,2.6772367,3.9,5.533333,13.41,growth_portfolio_return_percentage
2005,3.3927468,3.5,5.066667,7.68,growth_portfolio_return_percentage


In [10]:
final_dataframe<-collect(project_ds)
write_csv(final_dataframe,"final_data.csv")


## Methods
___

* We will conduct our data analysis by first dividing our data into **training data** and **testing data**. The class is known for our training data which includes the observations. This will be used as a basis for prediction for our classifier. Using the classifier, we can predict the class for the testing data whose classes are unknown.
* We’ll be using the **K-Nearest Neighbor Classification algorithm** from the parsnip R package in tidymodels, in order to make predictions.
    * We will use cross-validation to derive the best value to use as k.
    * We will be defining a model specification for the K-nearest neighbour and fit the model on the data by passing the model specification and data set to a fit function.
    * In the same step, we will specify the target variables (i.e, investment portfolio) and predictors that we are going to use (GDP, Inflation and Unemployment.)
* Finally, we’ll use the predict function to predict the best investment portfolio.
* We will visualize the result by:
    * Plotting a line graph with Returns in the y-axis and Time in the x-axis and factored by Portfolio.
    * If data permits, we will also produce either a KNN or OLS regression in order to predict annual returns based on the mentioned explanatory variables. The type of regression used will be determined by whichever provides the lowest RMSPE.


## Expected outcomes and significance
___

We expect to find which investment portfolio would provide you with the highest returns for a given set of assumptions/expectations about annual US unemployment rates, inflation rates, and real GDP growth rates; these are our explanatory variables of choice. After building the knn classification algorithm that provides the best portfolio under specific inflation, real GDP, and unemployment assumptions, we will then be able to plot the return of each of the six portfolios against each explanatory variable; this process involves graphing. Our findings would demonstrate the relationship each variable has with each portfolio, associating the impact of inflation, real GDP, and unemployment on the aforementioned investment portfolios. Having this association will provide us with the ability to make educated guesses regarding the future performance of a given portfolio. The ability to predict the outcome of portfolios presents us with many questions. For instance, are there any other portfolios that would perform better in different economic environments, and what composition would yield the highest returns? What would our model predict if we used portfolios that use more complex financial products beyond mutual funds or exchange traded funds?